In [ ]:
pip install pandas numpy faker

In [4]:
#import necessary libraries
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta, time
import os

#create a Faker instance
fake = Faker()


In [3]:
# Number of users and tasks
NUM_USERS = 10
TASKS_PER_USER = 20

# Priority levels
PRIORITY_LEVELS = ['Low', 'Medium', 'High']


In [14]:
"""
Generate a highly realistic task dataset (Option B)
- Period: 2024-07-01 to 2024-12-31 (6 months)
- Rows: 10,000
- Business hours: 9:00 - 17:00 (Mon-Fri)
- Team-specific patterns:
    - Engineering: steady + release spikes (Sept, Nov)
    - Marketing: campaign spikes (Oct, Nov)
    - Finance: month-end spikes (1st-3rd and 25th-31st)
    - Operations: steady daily work
    - Customer Support: frequent small tasks on weekdays
- Output CSV: data/raw/tasks_dataset_optionB_10k.csv
"""

import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, time
import os

# -----------------------------
# CONFIG
# -----------------------------
NUM_ROWS = 10000
START_DATE = datetime(2024, 7, 1)
END_DATE = datetime(2024, 12, 31)

WORK_START_HOUR = 9
WORK_END_HOUR = 17  # exclusive end (we will pick hours in [9..16] and minutes [0..59])

# -----------------------------
# TEAMS, ROLES, TASKS
# -----------------------------
teams = {
    "Engineering": {
        "roles": ["Backend Engineer", "Frontend Engineer", "Data Engineer", "QA Engineer"],
        "tasks": [
            "Fix backend API bug", "Implement new endpoint", "Database schema update",
            "Write frontend feature", "Refactor UI component", "Data pipeline monitoring",
            "ETL batch debugging", "Write unit tests", "Integration testing", "Performance profiling"
        ]
    },
    "Marketing": {
        "roles": ["Content Writer", "SEO Specialist", "Social Media Manager"],
        "tasks": [
            "Write blog post", "Keyword research", "Schedule social posts",
            "Email campaign setup", "Ad campaign optimization", "Competitor research",
            "Landing page copy", "Prepare campaign assets"
        ]
    },
    "Finance": {
        "roles": ["Accountant", "Financial Analyst"],
        "tasks": [
            "Prepare monthly report", "Reconcile transactions",
            "Create budget forecast", "Variance analysis", "Invoice processing",
            "Month-end close preparation"
        ]
    },
    "Operations": {
        "roles": ["Operations Associate", "Logistics Coordinator"],
        "tasks": [
            "Inventory check", "Vendor follow-up", "Shipment scheduling",
            "Process documentation", "Resource planning", "Warehouse audit"
        ]
    },
    "Customer Support": {
        "roles": ["Support Agent", "Technical Support"],
        "tasks": [
            "Respond to ticket", "Resolve technical issue", "Customer follow-up",
            "Escalate request", "Live chat assistance", "Create KB article"
        ]
    }
}

# -----------------------------
# Helper: priority rule
# -----------------------------
def assign_priority(task):
    t = task.lower()
    if any(k in t for k in ["bug", "error", "issue", "urgent", "fix", "debug"]):
        return "High"
    if any(k in t for k in ["report", "analysis", "campaign", "update", "planning", "month-end", "close"]):
        return "Medium"
    return "Low"

# -----------------------------
# Helper: business day/time utilities
# -----------------------------
def is_weekday(dt):
    return dt.weekday() < 5  # Mon-Fri => 0-4

def random_time_in_work_hours(date):
    hour = random.randint(WORK_START_HOUR, WORK_END_HOUR - 1)
    minute = random.randint(0, 59)
    return datetime.combine(date.date(), time(hour, minute))

def add_working_hours(start_dt, hours_to_add):
    """
    Add working hours to start_dt, skipping nights and weekends.
    Returns completed datetime.
    """
    current = start_dt
    remaining = hours_to_add
    while remaining > 0:
        # if weekend, move to next Monday 9:00
        if not is_weekday(current):
            days_to_monday = (7 - current.weekday())
            current = datetime.combine((current + timedelta(days=days_to_monday)).date(), time(WORK_START_HOUR, 0))
            continue

        end_of_day = datetime.combine(current.date(), time(WORK_END_HOUR, 0))
        hours_left_today = (end_of_day - current).total_seconds() / 3600.0
        if hours_left_today <= 0:
            # move to next workday start
            current = datetime.combine(current.date() + timedelta(days=1), time(WORK_START_HOUR, 0))
            continue

        if remaining <= hours_left_today:
            return current + timedelta(hours=remaining)
        else:
            remaining -= hours_left_today
            # move to next workday start
            current = datetime.combine(current.date() + timedelta(days=1), time(WORK_START_HOUR, 0))
    return current

# -----------------------------
# Seasonality / team-pattern sampler
# -----------------------------
def sample_date_for_team(team):
    """
    Sample a date between START_DATE and END_DATE influenced by team-specific patterns:
    - Engineering: steady + release spikes (Sept, Nov)
    - Marketing: campaign spikes (Oct, Nov)
    - Finance: month-end spikes (1..3 and 25..end)
    - Operations: uniform weekdays
    - Customer Support: uniform weekdays but higher daily frequency
    """
    # pick a base random day in range
    total_days = (END_DATE - START_DATE).days
    # bias selection using weights per month for certain teams
    while True:
        offset = random.randint(0, total_days)
        candidate = START_DATE + timedelta(days=offset)
        if not is_weekday(candidate):
            # prefer weekdays
            continue

        month = candidate.month

        if team == "Engineering":
            # more likely in Sept (9) and Nov (11)
            if month in (9, 11):
                if random.random() < 0.6:
                    return candidate
            # otherwise accept with 70% probability
            if random.random() < 0.7:
                return candidate

        elif team == "Marketing":
            # big campaign months Oct(10), Nov(11)
            if month in (10, 11):
                if random.random() < 0.7:
                    return candidate
            if random.random() < 0.5:
                return candidate

        elif team == "Finance":
            # shift probability to month-end: days 25-31 and 1-3
            dom = candidate.day
            if dom >= 25 or dom <= 3:
                if random.random() < 0.8:
                    return candidate
            if random.random() < 0.4:
                return candidate

        elif team == "Operations":
            # fairly uniform
            if random.random() < 0.85:
                return candidate

        elif team == "Customer Support":
            # uniform but higher frequency - accept more often
            if random.random() < 0.9:
                return candidate

        # fallback: accept occasionally
        if random.random() < 0.1:
            return candidate

# -----------------------------
# Build dataset
# -----------------------------
records = []
user_count = 200  # number of unique users to assign tasks to
task_counter = 1

# Precompute team base probabilities (we will use these to pick team)
team_names = list(teams.keys())
base_team_weights = {"Engineering":0.5, "Marketing":0.15, "Finance":0.1, "Operations":0.15, "Customer Support":0.1}
team_probs = [base_team_weights[t] for t in team_names]

for _ in range(NUM_ROWS):
    # choose team with base probabilities
    team = random.choices(team_names, weights=team_probs, k=1)[0]
    role = random.choice(teams[team]["roles"])
    task_name = random.choice(teams[team]["tasks"])
    description = f"{task_name} for {team.lower()} team by {role.lower()}."
    priority = assign_priority(task_name)
    user_id = random.randint(1, user_count)

    # Determine created_at influenced by team patterns / seasonality
    created_date = sample_date_for_team(team)
    created_at = random_time_in_work_hours(created_date)

    # determine expected duration (human estimate) depending on task type & team
    # base expected: shorter for support, variable for others
    if team == "Customer Support":
        expected = random.randint(1, 4)  # quick tickets
    elif team == "Marketing":
        expected = random.randint(2, 10)
    elif team == "Finance":
        expected = random.randint(2, 8)
    elif team == "Operations":
        expected = random.randint(1, 8)
    else:  # Engineering
        expected = random.randint(2, 12)

    # Simulate "true" completion hours (actual) with some noise and occasional overruns
    # base multiplier and noise
    base_multiplier = 1.1 if priority == "High" else 1.25 if priority == "Medium" else 1.35
    # more variance for engineering and marketing
    variance = 2.0 if team in ("Engineering","Marketing") else 1.2
    actual_hours = max(0.5, np.random.normal(expected * base_multiplier, variance))
    # occasionally inject big overrun (5% chance)
    if random.random() < 0.05:
        actual_hours *= random.uniform(1.5, 3.0)

    # compute completed_at respecting working hours / weekends
    completed_at = add_working_hours(created_at, actual_hours)

    # compute completion_time in working hours (rounded)
    completion_time = round((completed_at - created_at).total_seconds() / 3600.0, 2)

    records.append({
        "task_id": task_counter,
        "team": team,
        "role": role,
        "user_id": user_id,
        "task_name": task_name,
        "description": description,
        "priority": priority,
        "expected_duration": expected,
        "created_at": created_at,
        "completed_at": completed_at,
        "completion_time": completion_time
    })
    task_counter += 1

# -----------------------------
# Save CSV
# -----------------------------
df = pd.DataFrame.from_records(records)

# Ensure types and sorting
df['created_at'] = pd.to_datetime(df['created_at'])
df['completed_at'] = pd.to_datetime(df['completed_at'])
df = df.sort_values('created_at').reset_index(drop=True)

os.makedirs("data/raw", exist_ok=True)

df.to_csv('../data/raw/tasks_dataset.csv', index=False)
print(f"Saved {len(df)} rows to {out_path}")
print(df.head(10))


Saved 10000 rows to data/raw/tasks_dataset.csv
   task_id         team                   role  user_id  \
0     7908  Engineering          Data Engineer      133   
1     2251   Operations  Logistics Coordinator        3   
2     2178    Marketing   Social Media Manager       43   
3     3587  Engineering       Backend Engineer      133   
4     8602  Engineering            QA Engineer      108   
5     7164  Engineering       Backend Engineer      125   
6     2092   Operations  Logistics Coordinator      159   
7     9892  Engineering          Data Engineer       77   
8     9008  Engineering      Frontend Engineer      164   
9     3112  Engineering       Backend Engineer       98   

                task_name                                        description  \
0   Performance profiling  Performance profiling for engineering team by ...   
1       Resource planning  Resource planning for operations team by logis...   
2        Keyword research  Keyword research for marketing team 

In [10]:
df.head(20)

,task_id,team,role,user_id,task_name,description,priority,expected_duration,created_at,completed_at,completion_time
0,2995,Engineering,Backend Engineer,83,Fix backend API bug,Fix backend API bug for engineering team by ba...,High,10,2024-07-01 09:02:00,2024-07-02 09:41:00.918891,24.65
1,6560,Operations,Logistics Coordinator,6,Resource planning,Resource planning for operations team by logis...,Medium,8,2024-07-01 09:07:00,2024-07-02 10:11:18.579908,25.07
2,809,Operations,Operations Associate,43,Resource planning,Resource planning for operations team by opera...,Medium,4,2024-07-01 09:08:00,2024-07-01 15:42:12.603054,6.57
3,3944,Engineering,Data Engineer,147,Integration testing,Integration testing for engineering team by da...,Low,2,2024-07-01 09:12:00,2024-07-01 12:34:53.343184,3.38
4,2966,Engineering,Frontend Engineer,93,Performance profiling,Performance profiling for engineering team by ...,Low,10,2024-07-01 09:30:00,2024-07-02 13:54:48.032408,28.41
5,9779,Finance,Accountant,111,Prepare monthly report,Prepare monthly report for finance team by acc...,Medium,4,2024-07-01 09:43:00,2024-07-01 15:15:56.747906,5.55
6,4263,Customer Support,Support Agent,45,Create KB article,Create KB article for customer support team by...,Low,2,2024-07-01 10:02:00,2024-07-01 12:45:04.303907,2.72
7,5749,Engineering,QA Engineer,182,Database schema update,Database schema update for engineering team by...,Medium,3,2024-07-01 10:02:00,2024-07-01 15:51:45.556438,5.83
8,2024,Operations,Operations Associate,192,Resource planning,Resource planning for operations team by opera...,Medium,6,2024-07-01 10:17:00,2024-07-02 11:31:38.075079,25.24
9,1753,Engineering,QA Engineer,44,ETL batch debugging,ETL batch debugging for engineering team by qa...,High,11,2024-07-01 10:25:00,2024-07-02 10:57:00.093390,24.53


In [40]:
df.shape

(5000, 9)